This code tests whether the implemented function is actually working properly or not

It does this by calculating the moments over a unit cube with vertices (0,0,0) and (1,1,1) and edges aligned with axes and comparing with the analytical exact solution of the moments which is:

1/((i+1)*(j+1)*(k+1))

In [518]:
import pymeshlab
import numpy as np

In [519]:
m = pymeshlab.Mesh(
    vertex_matrix = [
        [0, 0, 0],
        [0, 0, 1],
        [0, 1 ,0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1 ,0],
        [1, 1 ,1]
    ],
    face_matrix = [
        [0,2,4],
        [4,2,6],
        [0,1,3],
        [0,3,2],
        [0,5,1],
        [0,4,5],
        [1,7,3],
        [1,5,7],
        [4,7,5],
        [4,6,7],
        [7,6,2],
        [7,2,3]
    ])

In [520]:
v_matrix = m.vertex_matrix()

In [521]:
f_matrix = m.face_matrix()

In [522]:
np.array(f_matrix)+1

array([[1, 3, 5],
       [5, 3, 7],
       [1, 2, 4],
       [1, 4, 3],
       [1, 6, 2],
       [1, 5, 6],
       [2, 8, 4],
       [2, 6, 8],
       [5, 8, 6],
       [5, 7, 8],
       [8, 7, 3],
       [8, 3, 4]], dtype=int32)

By default faces are defined and stored in the counter-clockwise order, so should not have any issues calculating the moments

In [523]:
# Define the maximum order of moments to be calculated
max_m = 5

In [524]:
def C_ijk(C,i,j,k):
    return (C[0]**i)*(C[1]**j)*(C[2]**k)*np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))
        

In [525]:
def D_ijk(D_tensor,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # D_ijk=0
        pass
    elif (0==i) and (0==j) and (0==k):
        # D_ijk = 1
        D_tensor[i,j,k] = 1
    else:
        D_tensor[i,j,k] = B[0]*D_tensor[i-1,j,k]+B[1]*D_tensor[i,j-1,k]+B[2]*D_tensor[i,j,k-1]+C_ijk(C,i,j,k)
    

In [526]:
def S_ijk(S_tensor,D_tensor,A,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # S_ijk = 0
        pass
    elif (0==i) and (0==j) and (0==k):
        # S_ijk = 1
        S_tensor[i,j,k] = 1
    else:
        S_tensor[i,j,k] = A[0]*S_tensor[i-1,j,k]+A[1]*S_tensor[i,j-1,k]+A[2]*S_tensor[i,j,k-1]+D_tensor[i,j,k]
    

In [527]:
def vol(A,B,C):
    return 1./6*np.linalg.det([A,B,C])


In [528]:
def M_ijk(f_matrix, v_matrix, max_m):
    M_tensor = np.zeros([max_m,max_m,max_m])
    
    
    for face_num in range(len(f_matrix)):
        
        
        # Get the three coordinates A,B,C
        # Any mesh library should return these in CCW order
        [A,B,C] = v_matrix[f_matrix[face_num]]
        
        # Calculate the face moments tensor
        # Each face moments tensor requires calculating S_ijk, D_ijk, and C_ijk Tensor Calculations.

        # C_ijk tensor is not reused, calculate on the fly
        
        # D_ijk tensor is reused, calculate and store
        D_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        D_ijk(D_tensor,B,C,i,j,k)
                        
        # S_ijk tensor is reqused, calculate and store
        S_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        S_ijk(S_tensor,D_tensor,A,B,C,i,j,k)
                        
        # Now calculate the moments on current face
        f_M_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        f_M_tensor[i,j,k] = (np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3)*np.linalg.det([A,B,C])*S_tensor[i,j,k]
        
        # Add contribution to the total moments
        M_tensor = M_tensor+f_M_tensor
                
    return M_tensor



In [529]:
moments = M_ijk(f_matrix,v_matrix,max_m)

In [530]:
test_tensor = np.zeros(np.shape(moments))

In [531]:
for i in range(max_m):
    for j in range(max_m):
        for k in range(max_m):
            if (i+j+k)<=max_m:
                test_tensor[i,j,k] = 1/((i+1)*(j+1)*(k+1))

In [532]:
np.linalg.norm(test_tensor-moments)

1.7053311162424517e-16

Since the value is ~ machine precision, the code is working properly!

In [533]:
print(moments)

[[[1.         0.5        0.33333333 0.25       0.2       ]
  [0.5        0.25       0.16666667 0.125      0.1       ]
  [0.33333333 0.16666667 0.11111111 0.08333333 0.        ]
  [0.25       0.125      0.08333333 0.         0.        ]
  [0.2        0.1        0.         0.         0.        ]]

 [[0.5        0.25       0.16666667 0.125      0.1       ]
  [0.25       0.125      0.08333333 0.0625     0.        ]
  [0.16666667 0.08333333 0.05555556 0.         0.        ]
  [0.125      0.0625     0.         0.         0.        ]
  [0.1        0.         0.         0.         0.        ]]

 [[0.33333333 0.16666667 0.11111111 0.08333333 0.        ]
  [0.16666667 0.08333333 0.05555556 0.         0.        ]
  [0.11111111 0.05555556 0.         0.         0.        ]
  [0.08333333 0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.25       0.125      0.08333333 0.         0.        ]
  [0.125      0.0625     0.         0.         0. 

In [541]:
v_matrix = np.array(v_matrix)

# Calculate the centered and scaled moments
v_matrix[:,0] = ( v_matrix[:,0] - moments[1,0,0]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
v_matrix[:,1] = ( v_matrix[:,1] - moments[0,1,0]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
v_matrix[:,2] = ( v_matrix[:,2] - moments[0,0,1]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
moments = M_ijk(f_matrix,v_matrix,max_m)
print(moments)

# Note how running this twice gives nicer results!

[[[ 1.00000000e+00  0.00000000e+00  8.33333333e-02  0.00000000e+00
    1.25000000e-02]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 8.33333333e-02  0.00000000e+00  6.94444444e-03  0.00000000e+00
    0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 1.25000000e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00 -4.33680869e-19  0.00000000e+00
    0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [-4.33680869e-19  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00]]

 [[ 8.33333333e-02  0.00000000e+00  6.94444444e-03  0.00000000e+00
    0.00000000e+00]
  [-4.33680869e-19  0.00000000e+00  0.0